In [ ]:
import rasterio
import numpy as np
from rasterio.enums import Resampling
from rasterio import features
from rasterio.transform import from_origin

def aggregate_raster_to_target_resolution(input_raster, output_raster, target_resolution):
    """
    Aggregates a raster to a new resolution by averaging.

    Parameters:
        input_raster (str): Path to the input raster file.
        output_raster (str): Path to the output aggregated raster file.
        target_resolution (int): The desired resolution (in meters) for the output raster.
    """
    # Open the input raster file
    with rasterio.open(input_raster) as src:
        # Read the data
        data = src.read(1)  # Assume we are working with a single-band raster
        transform = src.transform
        crs = src.crs
        width = src.width
        height = src.height

        # Calculate the number of pixels to aggregate (in both x and y directions)
        original_resolution = transform[0]  # This is the pixel size in the x-direction (meters)
        aggregation_factor = int(target_resolution / original_resolution)

        # Check if the target resolution is divisible by the original resolution
        if target_resolution % original_resolution != 0:
            raise ValueError(f"Target resolution {target_resolution} is not a multiple of the original resolution {original_resolution}")

        # Resample using aggregation (mean) for each target pixel (5km x 5km block)
        # First, reshape the data to aggregate it
        aggregated_data = data.reshape(
            (height // aggregation_factor, aggregation_factor, width // aggregation_factor, aggregation_factor)
        ).mean(axis=(1, 3))  # Average along the reshaped axes

        # Calculate the new transform for the output raster
        new_transform = from_origin(
            transform[2],  # Top left x (longitude)
            transform[5],  # Top left y (latitude)
            target_resolution,  # Target resolution in x
            target_resolution,  # Target resolution in y
        )

        # Define the new metadata for the output raster
        new_meta = src.meta.copy()
        new_meta.update({
            'driver': 'GTiff',
            'count': 1,  # Single band
            'dtype': 'float32',  # You may want to change this depending on the range of LST values
            'width': aggregated_data.shape[1],
            'height': aggregated_data.shape[0],
            'crs': crs,
            'transform': new_transform,
        })

        # Save the aggregated raster
        with rasterio.open(output_raster, 'w', **new_meta) as dst:
            dst.write(aggregated_data, 1)

    print(f"Aggregation complete! The aggregated raster is saved at {output_raster}.")

# Example usage:
input_raster = 'path_to_your_ecostress_lst_raster.tif'  # Path to the input raster
output_raster = 'aggregated_ecostress_lst_5km.tif'  # Path to the output raster
target_resolution = 5000  # 5 km in meters

aggregate_raster_to_target_resolution(input_raster, output_raster, target_resolution)
